In [1]:
import pandas as pd
from collections import Counter
from glob import glob
import csv
import os.path as path

In [2]:
from pie_ex_processortended.cli.utils import get_tagger
from pie_extended.models.dum.imports import get_iterator_and

In [87]:
n = 50
#path_name = 'test/*.txt'
path_name = 'txt/*.txt'

In [43]:
#_____________ PIE-EXTENDED  _________________

In [44]:
model_name = "dum"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

2021-11-27 14:30:52,347 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/lemma.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.
2021-11-27 14:30:53,117 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/pos.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.


In [74]:
def main(path_name, doc_name, n):
    with open(path_name, encoding="utf8") as file:
        lignes = file.readlines()

        for ligne in lignes:
            iterator, processor = get_iterator_and_processor()
            annotation = tagger.tag_str(ligne, iterator=iterator, processor=processor)
            list_three_chars = three_chars(ligne)

    df_pie = pd.DataFrame(annotation)
    
    df_tri_pos = list_freq_tri_pos(df_pie['pos'], n, doc_name)
         
    df_tri_char = list_freq_tri_char(list_three_chars, n, doc_name)
    
    list_lemma = nettoie_lemma(df_pie['lemma'], df_pie['pos'])
    df_lemma = list_freq_lemma(list_lemma, n, doc_name)

    return df_lemma, df_tri_pos, df_tri_char

In [75]:
def nettoie_lemma(list_lemma, list_pos):
    pos_ok = ["pre", "adv", "det", "conj", "pron", "adp", "art"]
    list_nettoie = []
    if len(list_lemma) == len(list_pos):
        for i in range(0, len(list_lemma)-1):
            if list_pos[i].split("(")[0] in pos_ok:
                list_nettoie.append(list_lemma[i])
    return list_nettoie

In [76]:
def list_freq_lemma(list_lemma, n, doc_name):
    table_freq_lemma = dict(Counter(list_lemma).most_common(n))
    
    zipped_lemma = list(zip(table_freq_lemma.keys(), table_freq_lemma.values()))
    df_lemma = pd.DataFrame(zipped_lemma, columns=['Lemma', 'Nombre d\'apparitions'])
    df_lemma['Frequence d\'apparition'] = round(df_lemma['Nombre d\'apparitions']/sum(table_freq_lemma.values()),5)
    
    df_lemma.sort_values('Frequence d\'apparition',ascending=False).to_csv(r'freq_lemma/'+doc_name+'_lemma_freq.csv', index = False)
    return df_lemma

In [77]:
def list_freq_tri_char(list_three_chars, n, doc_name):
    table_freq_chars = dict(Counter(list_three_chars).most_common(n))
    
    zipped_chars = list(zip(table_freq_chars.keys(), table_freq_chars.values()))
    df_chars = pd.DataFrame(zipped_chars, columns=['chars', 'Nombre d\'apparitions'])
    df_chars['Frequence d\'apparition'] = round(df_chars['Nombre d\'apparitions']/sum(table_freq_chars.values()),5)
    
    df_chars.sort_values('Frequence d\'apparition',ascending=False).to_csv(r'three_chars/'+doc_name+'_tri_char_freq.csv', index = False)
    return df_chars

In [78]:
def three_chars(chaine):
    list_three_chars = []
    for i in range(0, len(chaine)-2):
        list_three_chars.append(chaine[i]+chaine[i+1]+chaine[i+2])   
    return list_three_chars

In [79]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token].split("(")[0]+'_'+list_token[indice_token+1].split("(")[0]+'_'+list_token[indice_token+2].split("(")[0]
        list_trigram.append(trigram)
    return list_trigram

In [80]:
def list_freq_tri_pos(list_token, n, doc_name):
    list_trigram = trigrammize(list_token)
    table_freq_tri_pos = dict(Counter(list_trigram).most_common(n))
    zipped_tri_pos = list(zip(table_freq_tri_pos.keys(), table_freq_tri_pos.values()))
    df_tri_pos = pd.DataFrame(zipped_tri_pos, columns=['chars', 'Nombre d\'apparitions'])
    df_tri_pos['Frequence d\'apparition'] = round(df_tri_pos['Nombre d\'apparitions']/sum(table_freq_tri_pos.values()),5)
    
    df_tri_pos.sort_values('Frequence d\'apparition',ascending=False).to_csv(r'trigram_pos/'+doc_name+'_tri_pos_freq.csv', index = False)
    
    
    return df_tri_pos

In [81]:
def moulinette(path_name, n):
    df_main = pd.DataFrame()
    for doc in glob(path_name):
        doc_name = path.splitext(path.basename(doc))[0]
        df_lemma, df_tri_pos, df_tri_char = main(doc, doc_name, n)
        list_df = [df_lemma, df_tri_pos, df_tri_char]
        df_main[doc_name] = list_df
    return df_main

In [88]:
df_main = moulinette(path_name, n)

In [160]:
df_main["193536"][1]

,chars,Nombre d'apparitions,Frequence d'apparition
0,n_pron_n,5,0.07042
1,n_v_pron,4,0.05634
2,n_v_adp,2,0.02817
3,v_adp_n,2,0.02817
4,pron_n_conj,2,0.02817
5,v_pron_pron,2,0.02817
6,v_interj_v,2,0.02817
7,v_pron_adj,2,0.02817
8,pron_n_v,2,0.02817
9,v_v_n,2,0.02817


In [97]:
df_main["193536"][0]["Frequence d\'apparition"]

0     0.11290
1     0.08065
2     0.04839
3     0.04839
4     0.04839
5     0.04839
6     0.04839
7     0.03226
8     0.03226
9     0.03226
10    0.03226
11    0.03226
12    0.03226
13    0.01613
14    0.01613
15    0.01613
16    0.01613
17    0.01613
18    0.01613
19    0.01613
20    0.01613
21    0.01613
22    0.01613
23    0.01613
24    0.01613
25    0.01613
26    0.01613
27    0.01613
28    0.01613
29    0.01613
30    0.01613
31    0.01613
32    0.01613
33    0.01613
34    0.01613
35    0.01613
Name: Frequence d'apparition, dtype: float64

In [138]:
n = 200
path_name = 'rand/*.txt'
#path_name = 'txt/*.txt'

In [139]:
def moulinette(path_name, n):
    df_main = pd.DataFrame()
    for doc in glob(path_name):
        doc_name = path.splitext(path.basename(doc))[0]
        df_lemma, df_tri_pos, df_tri_char = main(doc, doc_name, n)
        list_df = [df_lemma, df_tri_pos, df_tri_char]
        df_main[doc_name] = list_df
    return df_main

In [140]:
df_rand = moulinette(path_name, n)

In [147]:
df_rand["2218"][2]["chars"]

0       mi
1      ijn
2      mij
3      jn 
4       de
      ... 
195    sli
196    ijt
197    jt 
198    n t
199     ti
Name: chars, Length: 200, dtype: object

In [136]:
os.getcwd()

'/home/humanum/Documents/proceeded/rand'

In [121]:
import os

In [150]:
os.chdir('rand/')
#os.chdir('/home/humanum/Documents/proceeded')

In [182]:
def df_to_list_dict(n):
    list_finale = []
    for doc in os.listdir():
        doc_name = path.splitext(path.basename(doc))[0]
        
        keys = df_rand[doc_name][n]['Lemma'].to_list() # Lemma / chars
        values = df_rand[doc_name][n]["Frequence d\'apparition"].to_list() 
        
        dict_temp = dict(zip(keys, values))
        
        list_finale.append(dict_temp)
        
    return list_finale

In [181]:
list_dict_tri_char = df_to_list_dict(2)
list_dict_tri_pos = df_to_list_dict(1)

In [183]:
list_dict_lemma = df_to_list_dict(0)

In [186]:
def concat_dict(list_dict):
    dict_concat = Counter()

    for dict_temp in list_dict:

        dict_concat += Counter(dict_temp)
    return dict_concat

In [190]:
dict_tri_char_term = concat_dict(list_dict_tri_char)
dict_tri_pos_term = concat_dict(list_dict_tri_pos)
dict_lemma_term = concat_dict(list_dict_lemma)

In [196]:
list_tri_char = list(dict(dict_tri_char_term.most_common(50)).keys())
list_tri_pos = list(dict(dict_tri_pos_term.most_common(50)).keys())
list_lemma = list(dict(dict_lemma_term.most_common(50)).keys())

In [200]:
print(list_tri_char)

['en ', 'er ', 'et ', 'n, ', 'en,', 't, ', ' de', 'den', 'an ', 'de ', ' he', 'ijn', ' ge', 'ver', 'een', ' ve', 'at ', 'oor', 'n d', 'aer', ' en', 'jn ', ' da', 'sch', 'eer', 'der', 'ck ', 'cht', 'ie ', ' va', ', D', ' ee', ' be', ' me', 'nde', ' we', 'aar', 'n s', 'ten', ' in', 't d', ' di', 'iet', ' mi', 'in ', 't g', 'ghe', 'te ', ' al', ' En']


In [201]:
print(list_tri_pos)

['n_v_pron', 'pron_n_v', 'adp_pron_n', 'adp_art_n', 'v_pron_n', 'n_adv_v', 'n_adp_n', 'art_n_v', 'adv_v_pron', 'v_pron_v', 'n_pron_n', 'n_n_v', 'n_art_n', 'v_art_n', 'v_adv_v', 'n_v_v', 'n_conj_n', 'pron_v_pron', 'v_conj_pron', 'n_pron_v', 'art_n_n', 'pron_adv_v', 'v_adp_pron', 'n_adp_pron', 'n_conj_pron', 'pron_n_n', 'n_n_n', 'v_adp_n', 'n_v_adv', 'pron_n_pron', 'pron_v_v', 'v_pron_adv', 'art_n_adp', 'pron_n_adv', 'v_v_pron', 'n_v_conj', 'art_adj_n', 'adv_v_v', 'n_v_adp', 'pron_v_adp', 'adp_n_n', 'v_pron_pron', 'adp_n_v', 'adv_v_adv', 'conj_pron_v', 'pron_v_adv', 'adv_adv_v', 'n_n_pron', 'conj_pron_n', 'pron_pron_v']


In [202]:
print(list_lemma)

['een', 'de', 'ik', 'gij', 'hij', 'ne', 'in', 'mijn', 'van', 'het', 'uw', 'en', 'zijn', 'wij', 'zij', 'eten', 'niet', 'dat', 'te', 'met', 'op', 'die', 'haar', 'eer', 'voor', 'hebben', 'zo', 'al', 'als', 'aan', 'et', 'hoofd', 'na', 'tot', 'ook', 'door', 'hier', 'of', 'hoe', 'dan', 'geen', 'om', 'nu', 'wel', 'ons', 'haten', 'deze', 'wat', 'dit', 'bij']
